In [4]:
import os
from osgeo import gdal
import numpy as np
from gref4hsi.utils.colours import Image as Imcol
from gref4hsi.utils.gis_tools import GeoSpatialAbstractionHSI

In [5]:

ref_ortho_reshaped_path = os.path.join(r"D:\Specim\Missions\2022-08-31-Remoy", 'orthomosaic_cropped.tif')
match_composite_path = os.path.join(r"D:\Specim\Missions\2022-08-31-Remoy", 'orthomosaic_cropped_high_tide.tif')

do_something = False

if do_something:
    fname_low = os.path.join(r"C:\Users\haavasl\OneDrive - NTNU\GeoHab\Figures", '0800_full.tif')
    fname_mid = os.path.join(r"C:\Users\haavasl\OneDrive - NTNU\GeoHab\Figures", '1040_full.tif')
    fname_high = os.path.join(r"C:\Users\haavasl\OneDrive - NTNU\GeoHab\Figures", '1435_full.tif')

    # Coreg equivalent
    fname_low = os.path.join(r"C:\Users\haavasl\OneDrive - NTNU\GeoHab\Figures", '0800_full_coreg.tif')
    fname_mid = os.path.join(r"C:\Users\haavasl\OneDrive - NTNU\GeoHab\Figures", '1040_full_coreg.tif')
    fname_high = os.path.join(r"C:\Users\haavasl\OneDrive - NTNU\GeoHab\Figures", '1435_full_coreg.tif')

    ref_ortho_path = fname_low
    match_composite_path = fname_high

    ref_ortho_reshaped_path = os.path.join(r'C:\Users\haavasl', 'temp_tif.tif')

    print(ref_ortho_reshaped_path)
    os.remove(ref_ortho_reshaped_path)

    GeoSpatialAbstractionHSI.resample_rgb_ortho_to_hsi_ortho(ref_ortho_path, match_composite_path, ref_ortho_reshaped_path)


In [6]:
# The ref orthomosaic after reshaping (the reference)
raster_ref = gdal.Open(ref_ortho_reshaped_path, gdal.GA_Update)
xoff1, a1, b1, yoff1, d1, e1 = raster_ref.GetGeoTransform()  # This should be equal
raster_ref_array = np.array(raster_ref.ReadAsArray())
R = raster_ref_array[0, :, :].reshape((raster_ref_array.shape[1], raster_ref_array.shape[2], 1))
G = raster_ref_array[1, :, :].reshape((raster_ref_array.shape[1], raster_ref_array.shape[2], 1))
B = raster_ref_array[2, :, :].reshape((raster_ref_array.shape[1], raster_ref_array.shape[2], 1))
# del raster_array1
ortho_ref = np.concatenate((R, G, B), axis=2)
ref_image = Imcol(ortho_ref)

In [7]:
# The match composite raster (the match)
raster_match = gdal.Open(match_composite_path)
raster_match_array = np.array(raster_match.ReadAsArray())
xoff2, a2, b2, yoff2, d2, e2 = raster_match.GetGeoTransform()
transform_pixel_projected = raster_match.GetGeoTransform()
R = raster_match_array[0, :, :].reshape((raster_match_array.shape[1], raster_match_array.shape[2], 1))
G = raster_match_array[1, :, :].reshape((raster_match_array.shape[1], raster_match_array.shape[2], 1))
B = raster_match_array[2, :, :].reshape((raster_match_array.shape[1], raster_match_array.shape[2], 1))
ortho_match = np.concatenate((R, G, B), axis=2)
match_image = Imcol(ortho_match)

In [8]:
from gref4hsi.utils.gis_tools import GeoSpatialAbstractionHSI
# Do some image processing here
match_image.clahe_adjustment()
ref_image.clahe_adjustment()

ref_image.to_luma(gamma=False, image_array= ref_image.clahe_adjusted)
match_image.to_luma(gamma=False, image_array= match_image.clahe_adjusted)





uv_vec_match, uv_vec_ref, diff_AE_pixels = GeoSpatialAbstractionHSI.compute_sift_difference(match_image.luma_array, ref_image.luma_array)

18008


In [9]:
resolution = a1*100 # In cm

print(np.mean(diff_AE_pixels[diff_AE_pixels<100])*resolution)
print(np.median(diff_AE_pixels[diff_AE_pixels<100])*resolution)

err = (uv_vec_match[diff_AE_pixels<20] - uv_vec_ref[diff_AE_pixels<20])*resolution
med_shift = np.median(err, axis = 0)

print(err.shape[0])
err_shifted = err - med_shift

abs_err_shift = np.abs(err_shifted)
print(np.median(abs_err_shift))
#med_shift = np.median(uv_vec_match[diff_AE_pixels<10] - uv_vec_ref[diff_AE_pixels<10], axis = 0)
#print()


#print()

9.537728563699968
7.2058019233947865
12862
3.9807675895308217
